<a href="https://colab.research.google.com/github/shahdhesham/Thesis_Eval/blob/main/Consolidated_EVAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reset all runtime variables
%reset -f


In [ ]:
import os
import glob
import subprocess
import zipfile
from pathlib import Path
from google.colab import files


# CodeBlue

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/k4black/codebleu.git


Cloning into 'codebleu'...
remote: Enumerating objects: 761, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 761 (delta 205), reused 138 (delta 138), pack-reused 518 (from 2)
Receiving objects: 100% (761/761), 1.32 MiB | 5.80 MiB/s, done.
Resolving deltas: 100% (407/407), done.


In [ ]:
%cd codebleu
!pip install -r requirements.txt


/content/codebleu
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!mkdir -p /content/codebleu/keywords
!curl -o /content/codebleu/keywords/python.txt https://raw.githubusercontent.com/k4black/codebleu/main/keywords/python.txt
!curl -o /content/codebleu/keywords/cpp.txt https://raw.githubusercontent.com/k4black/codebleu/main/keywords/cpp.txt
!curl -o /content/codebleu/keywords/java.txt https://raw.githubusercontent.com/k4black/codebleu/main/keywords/java.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     67      0 --:--:-- --:--:-- --:--:--    68
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     65      0 --:--:-- --:--:-- --:--:--    65
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     55      0 --:--:-- --:--:-- --:--:--    55


In [ ]:
# !pip uninstall -y tree-sitter tree-sitter-python tree-sitter-cpp tree-sitter-java
!pip install git+https://github.com/k4black/codebleu.git#egg=codebleu[all]


DEPRECATION: git+https://github.com/k4black/codebleu.git#egg=codebleu[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/k4black/codebleu.git to /tmp/pip-install-9ae3k2m7/codebleu_fc4a4fc0dffa431a82307bb1a0e0a781
  Running command git clone --filter=blob:none --quiet https://github.com/k4black/codebleu.git /tmp/pip-install-9ae3k2m7/codebleu_fc4a4fc0dffa431a82307bb1a0e0a781
  Resolved https://github.com/k4black/codebleu.git to commit b0edb622f6a52fe9d1edc407be5061d3e1462a7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.6/567.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
from codebleu import calc_codebleu

# Code Eval

In [ ]:
# 📥 Step 1: Upload ZIP files that contain original C code and translated C++ code
print("Upload your ZIP file containing original C code (e.g., 5solutions.zip):")
uploaded = files.upload()                          # Upload the C ZIP file
c_zip = next(iter(uploaded))                       # Get uploaded file name

print("Upload your ZIP file containing translated C++ code (e.g., output_folder.zip):")
uploaded = files.upload()                          # Upload the C++ ZIP file
cpp_zip = next(iter(uploaded))                     # Get uploaded file name

# 📥 Step 1.3: Upload ZIP file containing reference C++ code (gold standard)
print("Upload your ZIP file containing reference C++ code (e.g., reference.zip):")
uploaded = files.upload()                          # Upload the reference ZIP
ref_zip = next(iter(uploaded))                     # Get uploaded file name



Upload your ZIP file containing original C code (e.g., 5solutions.zip):


Saving 5solutions.zip to 5solutions.zip
Upload your ZIP file containing translated C++ code (e.g., output_folder.zip):


Saving output_folder.zip to output_folder.zip
Upload your ZIP file containing reference C++ code (e.g., reference.zip):


Saving reference.zip to reference.zip


In [ ]:
# 📂 Step 2: Unzip both folders into the current working directory
with zipfile.ZipFile(c_zip, 'r') as zip_ref:
    zip_ref.extractall()                           # Extract C code folder

with zipfile.ZipFile(cpp_zip, 'r') as zip_ref:
    zip_ref.extractall()                           # Extract C++ code folder

    # 📂 Step 1.4: Unzip reference into the working directory
with zipfile.ZipFile(ref_zip, 'r') as zip_ref:
    zip_ref.extractall()                           # Extracts to 'reference/selected_100/...'


In [ ]:
ref_base = "reference/selected_100"
hyp_base = "output_folder/5solutions"
keywords_dir = Path("/content/codebleu/codebleu/keywords")
custom_weights = (0.1, 0.1, 0.3, 0.5)


In [ ]:
# # --- Compile and run a C or C++ file ---
# def compile_and_run(path, is_cpp=False):
#     exe = path.replace('/', '_').replace('.c', '').replace('.cpp', '') + "_exe"
#     cmd = ['gcc', path, '-o', exe]
#     if is_cpp:
#         cmd = ['gcc', '-x', 'c++', path, '-o', exe, '-lstdc++']
#     compiled = subprocess.run(cmd, capture_output=True, text=True)
#     if compiled.returncode != 0:
#         return False, None
#     try:
#         run = subprocess.run([f"./{exe}"], capture_output=True, text=True, timeout=30, errors="replace")
#     except subprocess.TimeoutExpired:
#         return True, None
#     except Exception as e:
#         return True, None
#     stdout = run.stdout.strip()
#     return True, stdout if run.returncode == 0 else None

In [ ]:
hyp_base = "output_folder/5solutions"  # Adjust if needed
print("hyp_base contents:", os.listdir(hyp_base))
problem_folders = sorted(glob.glob(os.path.join(hyp_base, "P*")))
print("problem_folders found:", problem_folders)

hyp_base contents: ['p00076', 'p00129', 'p00136', 'p01118', 'p02833', 'p00379', 'p03836', 'p02610', 'p03754', 'p00317', 'p03293', 'p02947', 'p00482', 'p02972', 'p02623', 'p00947', 'p00423', 'p00288', 'p01353', 'p01147', 'p01376', 'p02402', 'p01766', 'p03395', 'p00659', 'p03820', 'p00465', 'p03482', 'p02987', 'p03921', 'p02906', 'p02536', 'p01909', 'p03113', 'p01600', 'p03384', 'p03313', 'p00080', 'p03765', 'p01236', 'p03184', 'p02176', 'p01407', 'p02257', 'p01418', 'p00536', 'p00160', 'p00189', 'p00133', 'p01368', 'p02017', 'p02465', 'p03322', 'p01105', 'p00333', 'p02022', 'p01075', 'p01256', 'p03378', 'p01201', 'p00595', 'p00548', 'p03009', 'p03284', 'p03686', 'p00224', 'p02240', 'p00363', 'p01770', 'p02549', 'p00034', 'p01601', 'p02065', 'p00719', 'p02534']
problem_folders found: []


In [ ]:
import subprocess

def compile_and_run(path, is_cpp=False):
    """
    Compiles and runs a C or C++ file.
    Returns:
      - success (True/False)
      - output (if successful)
      - compile_type: 'C++' or the C standard used (e.g., 'c11', 'gnu89', etc.)
    """
    exe = path.replace('/', '_').replace('.c', '').replace('.cpp', '') + "_exe"

    if is_cpp:
        # C++ compilation (unchanged from your original)
        cmd = ['g++', path, '-o', exe]  # Changed from gcc -x c++ to g++
        compiled = subprocess.run(cmd, capture_output=True, text=True)
        if compiled.returncode != 0:
            print(f"❌ C++ compilation failed for {path}")
            print(f"   ↪️ Error: {compiled.stderr.strip()}")
            return False, None, 'C++ (failed)'
        compile_type = 'C++'
    else:
        # C compilation - IMPROVED VERSION
        standards = ['c11', 'c99', 'gnu89']  # Removed c89 (too strict for modern code)
        compile_type = None

        for std in standards:
            cmd = [
                'gcc',
                f'-std={std}',
                '-Wall',               # Enable warnings
                '-Wextra',             # Extra warnings
                path,
                '-o', exe,
                '-lm'                  # CRITICAL: Added math library linking
            ]
            compiled = subprocess.run(cmd, capture_output=True, text=True)

            if compiled.returncode == 0:
                compile_type = std
                print(f"✅ Compiled {path} with -std={std}")
                break
            else:
                print(f"⚠️ Failed -std={std} for {path}")
                print(f"   ↪️ Error: {compiled.stderr.strip()}")

        if not compile_type:
            return False, None, 'All C standards failed'

    # Execution handling (unchanged from your robust version)
    try:
        run = subprocess.run(
            [f"./{exe}"],
            capture_output=True,
            text=True,
            timeout=30,
            errors="replace"
        )
        stdout = run.stdout.strip()
        if run.returncode == 0:
            return True, stdout, compile_type
        else:
            return True, f"💥 Exit code {run.returncode}", compile_type

    except subprocess.TimeoutExpired:
        print(f"⏱️ Timeout for: {path}")
        return True, "⏱️ Timeout", compile_type
    except Exception as e:
        print(f"💥 Runtime error for: {path} → {e}")
        return True, f"💥 Runtime error: {e}", compile_type

In [ ]:
# import subprocess

# def compile_and_run(path, is_cpp=False):
#     """
#     Compiles and runs a C or C++ file.
#     Returns:
#       - success (True/False)
#       - output (if successful)
#       - compile_type: 'C++' or the C standard used (e.g., 'c99', 'gnu89', etc.)
#     """
#     exe = path.replace('/', '_').replace('.c', '').replace('.cpp', '') + "_exe"

#     # if is_cpp:
#     #     cmd = ['gcc', '-x', 'c++', path, '-o', exe, '-lstdc++']
#     #     compiled = subprocess.run(cmd, capture_output=True, text=True)
#     #     if compiled.returncode != 0:
#     #         return False, None, 'C++ (failed)'
#     #     compile_type = 'C++'
#     # else:
#     #     # Try multiple C standards in order of modern → legacy
#     #     standards = ['c11', 'c99', 'c89', 'gnu89']
#     #     compiled = None
#     #     compile_type = 'Unknown'
#     #     for std in standards:
#     #         cmd = ['gcc', f'-std={std}', path, '-o', exe]
#     #         compiled = subprocess.run(cmd, capture_output=True, text=True)
#     #         if compiled.returncode == 0:
#     #             compile_type = std
#     #             print(f"✅ Compiled {path} with -std={std}")
#     #             break
#     #         else:
#     #             print(f"⚠️ Failed -std={std} for {path}")
#     #     if compiled is None or compiled.returncode != 0:
#     #         return False, None, 'All C standards failed'


#     if is_cpp:
#       cmd = ['gcc', '-x', 'c++', path, '-o', exe, '-lstdc++']
#       compiled = subprocess.run(cmd, capture_output=True, text=True)
#       if compiled.returncode != 0:
#           print(f"❌ C++ compilation failed for {path}")
#           print(f"   ↪️ Error: {compiled.stderr.strip()}")
#           return False, None, 'C++ (failed)'
#       compile_type = 'C++'
#     else:
#       # Try multiple C standards in order of modern → legacy
#       standards = ['c11', 'c99', 'c89', 'gnu89']
#       compiled = None
#       compile_type = 'Unknown'
#       for std in standards:
#           cmd = ['gcc', f'-std={std}', path, '-o', exe]
#           compiled = subprocess.run(cmd, capture_output=True, text=True)
#           if compiled.returncode == 0:
#               compile_type = std
#               print(f"✅ Compiled {path} with -std={std}")
#               break
#           else:
#               print(f"⚠️ Failed -std={std} for {path}")
#               print(f"   ↪️ Error: {compiled.stderr.strip()}")
#       if compiled is None or compiled.returncode != 0:
#           return False, None, 'All C standards failed'


#     # Run executable
#     # try:
#     #     run = subprocess.run([f"./{exe}"], capture_output=True, text=True, timeout=30, errors="replace")
#     # except subprocess.TimeoutExpired:
#     #     print(f"⏱️ Timeout for: {path}")
#     #     return True, None, compile_type
#     # except Exception as e:
#     #     print(f"💥 Runtime error for: {path} → {e}")
#     #     return True, None, compile_type

#     # return True, run.stdout.strip() if run.returncode == 0 else None, compile_type


#     try:
#         run = subprocess.run([f"./{exe}"], capture_output=True, text=True, timeout=30, errors="replace")
#     except subprocess.TimeoutExpired:
#         print(f"⏱️ Timeout for: {path}")
#         return True, "⏱️ Timeout", compile_type
#     except Exception as e:
#         print(f"💥 Runtime error for: {path} → {e}")
#         return True, f"💥 Error: {e}", compile_type

#     # Handle normal run
#     stdout = run.stdout.strip()
#     return True, stdout if run.returncode == 0 else f"💥 Exit code {run.returncode}", compile_type


In [ ]:
# 📄 File Role Clarification
# - hyp_file   : The translated C++ file generated by the model (Hypothesis)
# - ref_files  : List of ground truth C++ reference files written by humans (References)
# - c_path     : The original C file that was translated (Input Source)
#
# These are used for:
# - CR & CA    : Comparing original C file to its translated C++ counterpart
# - CodeBLEU   : Comparing translated C++ (hypothesis) against all available reference C++ files


In [ ]:
# os.makedirs("detailed_output", exist_ok=True) Uncomment this


In [ ]:
# --- Evaluation loop ---
results = []
problem_folders = sorted(glob.glob(os.path.join(hyp_base, "p*")))

for p_path in problem_folders:
    folder_name = os.path.basename(p_path)
    ref_path = os.path.join(ref_base, folder_name, "C++", "*.cpp")
    hyp_path = os.path.join(p_path, "C", "*.cpp")

    ref_files = sorted(glob.glob(ref_path))
    hyp_files = sorted(glob.glob(hyp_path))

    if not hyp_files:
        print(f"⚠️ Skipping {folder_name} (No translated files)")
        continue

    print(f"📁 {folder_name} → {len(ref_files)} reference files found")

    # Load non-empty references
    ref_codes = []
    for r in ref_files:
        with open(r, "r", encoding="utf-8") as ref:
            content = ref.read().strip()
            if content:
                ref_codes.append(content)
            else:
                print(f"⚠️ Empty reference file skipped: {r}")

    for hyp_file in hyp_files:
        with open(hyp_file, "r", encoding="utf-8") as h:
            hyp_code = h.read()

        # CR & CA evaluation
        rel = os.path.relpath(hyp_file, hyp_base)
        c_path = os.path.join("5solutions", rel).replace(".cpp", ".c")
        if not os.path.exists(c_path):
            print(f"⚠️ C file not found for {hyp_file}, skipping...")
            continue

        c_ok, c_out, c_std = compile_and_run(c_path)
        cpp_ok, cpp_out, cpp_std = compile_and_run(hyp_file, is_cpp=True)

        # Run CodeBLEU only if reference codes exist
        if ref_codes:
            bleu_result = calc_codebleu(
                references=[ref_codes],
                predictions=[hyp_code],
                lang="cpp",
                keywords_dir=keywords_dir,
                weights=custom_weights
            )
        else:
            print(f"⚠️ Skipping CodeBLEU for {hyp_file} (no references)")
            bleu_result = "N/A"

    if c_out == "⏱️ Timeout":
        ca_result = "N/A"
    else:
        ca_result = cpp_ok and c_ok and (c_out == cpp_out)

        results.append({
            "C File": c_path,
            "C++ File": hyp_file,
            "CR": cpp_ok,
            "CA": ca_result,
            "C Output": c_out,
            "C Timed Out": c_out == "⏱️ Timeout",
            "C++ Timed Out": cpp_out == "⏱️ Timeout",
            "C++ Output": cpp_out,
            "C Standard": c_std,
            "CodeBLEU": bleu_result
        })

        print(f"✓ Evaluated: {c_path} ↔ {hyp_file}")


Streaming output truncated to the last 5000 lines.
      |                 ^
output_folder/5solutions/p01418/C/s859067641.cpp:17:20: error: ‘L’ was not declared in this scope
   17 |         cin>>K>>R>>L>>P>>E>>T;
      |                    ^
output_folder/5solutions/p01418/C/s859067641.cpp:17:23: error: ‘P’ was not declared in this scope
   17 |         cin>>K>>R>>L>>P>>E>>T;
      |                       ^
output_folder/5solutions/p01418/C/s859067641.cpp:17:26: error: ‘E’ was not declared in this scope
   17 |         cin>>K>>R>>L>>P>>E>>T;
      |                          ^
output_folder/5solutions/p01418/C/s859067641.cpp:17:29: error: ‘T’ was not declared in this scope
   17 |         cin>>K>>R>>L>>P>>E>>T;
      |                             ^
output_folder/5solutions/p01418/C/s859067641.cpp: At global scope:
output_folder/5solutions/p01418/C/s859067641.cpp:22:1: error: ‘Please’ does not name a type
   22 | Please note that the C++ code is more complex and contains more functional

In [ ]:
#Uncomment this
## --- Save results if any so ihave this level of detail at hand ---
# if results:
#     json_path = os.path.join("detailed_output", "evaluation_results.json")
#     csv_path = os.path.join("detailed_output", "evaluation_results.csv")

#     # Save JSON
#     with open(json_path, "w", encoding="utf-8") as jf:
#         json.dump(results, jf, indent=2)

#     # Save CSV
#     with open(csv_path, "w", newline="", encoding="utf-8") as cf:
#         writer = csv.DictWriter(cf, fieldnames=results[0].keys())
#         writer.writeheader()
#         writer.writerows(results)

#     # --- Zip the output folder ---
#     zip_path = "detailed_output.zip"
#     with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, _, files in os.walk("detailed_output"):
#             for file in files:
#                 full_path = os.path.join(root, file)
#                 arcname = os.path.relpath(full_path, start="detailed_output")
#                 zipf.write(full_path, arcname)

#     print(f"\n✅ Results saved in: {json_path}, {csv_path}")
#     print(f"📦 Zipped archive created: {zip_path}")

# else:
#     print("\n⚠️ No results were collected — nothing to save or zip.")


FileNotFoundError: [Errno 2] No such file or directory: 'detailed_output/evaluation_results.json'

In [ ]:
# --- 7. Summary ---
print(f"\n🔍 Total file pairs evaluated: {len(results)}")
cr_count = sum(1 for r in results if r['CR'])
ca_count = sum(1 for r in results if r['CA'])
c_timeouts = sum(r["C Timed Out"] for r in results)
cpp_timeouts = sum(r["C++ Timed Out"] for r in results)
ca_skipped = sum(1 for r in results if r["CA"] == "N/A")

print(f"✅ Compilation Rate (CR): {cr_count}/{len(results)}")
print(f"🤖 Computational Accuracy (CA): {ca_count}/{len(results)}")
print(f"⏱️ C Timeouts: {c_timeouts}")
print(f"⏱️ C++ Timeouts: {cpp_timeouts}")
print(f"🟡 CA Skipped due to C timeout: {ca_skipped}")

print("\n📄 Detailed Results:")
for r in results:
    print(f"→ {r['C File']} ↔ {r['C++ File']}")
    print(f"   CR: {r['CR']}, CA: {r['CA']}")
    print(f"   🧪 C Standard: {r['C Standard']}")

    # Handle cases where CodeBLEU is "N/A" or a dict
    print("   📊 CodeBLEU Scores:")
    if isinstance(r['CodeBLEU'], dict):  # Check if it's a dictionary
        for k, v in r['CodeBLEU'].items():
            print(f"      {k}: {v:.4f}")
    else:
        print(f"      {r['CodeBLEU']}")  # Prints "N/A" or other strings

    # Output comparison
    if r['CR'] and not r['CA']:
        print(f"   ❌ Mismatch Detected:")
        print(f"   🔸 C Output   : {r['C Output']}")
        print(f"   🔹 C++ Output : {r['C++ Output']}")
    else:
        print(f"   ✅ No mismatch detected.")
        print(f"   🔸 C Output   : {r['C Output']}")
        print(f"   🔹 C++ Output : {r['C++ Output']}")


🔍 Total file pairs evaluated: 305
✅ Compilation Rate (CR): 132/305
🤖 Computational Accuracy (CA): 96/305
⏱️ C Timeouts: 27
⏱️ C++ Timeouts: 5
🟡 CA Skipped due to C timeout: 0

📄 Detailed Results:
→ 5solutions/p00034/C/s084585556.c ↔ output_folder/5solutions/p00034/C/s084585556.cpp
   CR: True, CA: True
   🧪 C Standard: c11
   📊 CodeBLEU Scores:
      N/A
   ✅ No mismatch detected.
   🔸 C Output   : 
   🔹 C++ Output : 
→ 5solutions/p00034/C/s105671040.c ↔ output_folder/5solutions/p00034/C/s105671040.cpp
   CR: False, CA: False
   🧪 C Standard: c11
   📊 CodeBLEU Scores:
      N/A
   ✅ No mismatch detected.
   🔸 C Output   : 
   🔹 C++ Output : None
→ 5solutions/p00034/C/s512096282.c ↔ output_folder/5solutions/p00034/C/s512096282.cpp
   CR: True, CA: True
   🧪 C Standard: c11
   📊 CodeBLEU Scores:
      N/A
   ✅ No mismatch detected.
   🔸 C Output   : 
   🔹 C++ Output : 
→ 5solutions/p00034/C/s664006655.c ↔ output_folder/5solutions/p00034/C/s664006655.cpp
   CR: True, CA: True
   🧪 C Stan

In [ ]:
# --- Save Results ---
output_file = "evaluation_results.txt"
cr_count = sum(1 for r in results if r['CR'])
ca_count = sum(1 for r in results if r['CA'])

with open(output_file, "w", encoding="utf-8") as f:
    f.write("🔍 Summary of Evaluation\n")
    f.write(f"Total file pairs evaluated: {len(results)}\n")
    f.write(f"✅ Compilation Rate (CR): {cr_count}/{len(results)}\n")
    f.write(f"🤖 Computational Accuracy (CA): {ca_count}/{len(results)}\n\n")
    f.write("📄 Detailed Results\n")

    for r in results:
        f.write(f"→ {r['C File']} ↔ {r['C++ File']}\n")
        f.write(f"   CR: {r['CR']}, CA: {r['CA']}\n")

        if r['CR'] and not r['CA']:
            f.write("   ❌ Mismatch Detected:\n")
        else:
            f.write("   ✅ No mismatch detected.\n")

        f.write(f"   🔸 C Output   : {r['C Output']}\n")
        f.write(f"   🔹 C++ Output : {r['C++ Output']}\n")
        f.write(f"   🧪 C Standard: {r['C Standard']}\n")

        f.write("   📊 CodeBLEU Scores:\n")
        if isinstance(r['CodeBLEU'], dict):  # Check if it's a dictionary
            for k, v in r['CodeBLEU'].items():
                f.write(f"      {k}: {v:.4f}\n")
        else:
            f.write(f"      {r['CodeBLEU']}\n")  # Write the string (e.g., "N/A")

        f.write("\n")  # Add empty line between entries

In [ ]:

# --- Zip and download ---
zip_name = "evaluation_output.zip"
with zipfile.ZipFile(zip_name, 'w') as zipf:
    zipf.write(output_file)

from google.colab import files
files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>